In [1]:
from google.colab import files

uploaded = files.upload()
# Kokeile "lakko" -datoja

Saving reddit_matched_sample_test.parquet to reddit_matched_sample_test.parquet
Saving reddit_matched_sample_train.parquet to reddit_matched_sample_train.parquet


In [3]:
#!pip install pyarrow

In [4]:
import pandas as pd
import pyarrow

#df_train = pd.read_parquet("reddit_data_sample_train_s.parquet") # s on pienempi data, 15892 riviä
#df_eval = pd.read_parquet("reddit_data_sample_test_s.parquet")

df_train = pd.read_parquet("reddit_matched_sample_train.parquet")
df_eval = pd.read_parquet("reddit_matched_sample_test.parquet")

In [5]:
df_train["labels"] = df_train["labels"].astype("int")
df_eval["labels"] = df_eval["labels"].astype("int")

df_train["text"] = df_train["text"].astype("str")
df_eval["text"] = df_eval["text"].astype("str")

In [6]:
df_train.head()

,labels,text
0,0,"oatlyn iKaffe on maitoa parempaa kahvissa, en ..."
1,1,Huomasitko että Nyssen sivuilla ei lue että ra...
2,0,Helpoin tapa on tehdä asp tili ja säästää sill...
3,0,No jos luetaan asunnot pois niin velkaa 280k n...
4,0,"Mutta jos säästät aspille enemmän, niin voit t..."


In [7]:
df_eval.head()

,labels,text
0,0,"Jos haluat nimenomaan pörssiin sijoittaa, niin..."
1,1,Ohan pirkkalassa valtavat ostoskeskuksetkin. J...
2,1,Tuo /u/soskz :n linkkaaman kilke tosiaan viitt...
3,1,Ainakin tuossa Tamperelaisen artikkelissa main...
4,1,Paras pulkkamäki on tietysti Näyttämönpuiston ...


In [8]:
df_train.dtypes

,0
labels,int64
text,object


In [9]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [ ]:
#!pip install transformers

In [10]:
from datasets import Dataset

dataset_train = Dataset.from_pandas(df_train)
dataset_eval = Dataset.from_pandas(df_eval)

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

#import transformers
#import torch

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.3 MB/s eta 0:00:00


In [12]:
from peft import get_peft_model, LoraConfig, TaskType

peft_config = LoraConfig(
    inference_mode=False, lora_dropout=0.1
)

In [13]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
# output: trainable params: 294,912 || all params: 278,340,098 || trainable%: 0.10595383206339175


trainable params: 294,912 || all params: 278,340,098 || trainable%: 0.1060


In [14]:
from transformers import AutoTokenizer, DataCollatorWithPadding

def tokenize_function(example):
    return tokenizer(example["text"], padding=True, truncation=True) # Mieti erityisesti truncationia viestidatassa

tokenized_train = dataset_train.map(tokenize_function, batched=True)
tokenized_eval = dataset_eval.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/52245 [00:00<?, ? examples/s]

Map:   0%|          | 0/5733 [00:00<?, ? examples/s]

In [15]:
tokenized_train = tokenized_train.remove_columns("text") # POISTA TÄSSÄ VAIN TEXT!
tokenized_eval = tokenized_eval.remove_columns("text")

In [16]:
tokenized_train.set_format("torch")
tokenized_eval.set_format("torch")

In [17]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_train, shuffle = True, batch_size = 8, collate_fn=data_collator)
eval_dataloader = DataLoader(tokenized_eval, batch_size = 8, collate_fn=data_collator)

In [18]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 512]),
 'attention_mask': torch.Size([8, 512])}

In [19]:
import torch

torch.cuda.is_available()

True

In [20]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=1e-5) # Mitä lr tässä tarkoittaa? Learning rate?
# lr laskettu 5e-5 -> 1e-5

# Tutustu, miten määrittää oikea learning rate

In [21]:
from transformers import get_scheduler

num_epochs = 2 # Mieti tätä -> PIENENTÄMINEN VOI AUTTAA SAMOJEN AROVJEN ENNUSTAMISEEN, LASKE ESIM. KAHTEEN
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [22]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

PeftModel(
  (base_model): LoraModel(
    (model): XLMRobertaForSequenceClassification(
      (roberta): XLMRobertaModel(
        (embeddings): XLMRobertaEmbeddings(
          (word_embeddings): Embedding(250002, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): XLMRobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x XLMRobertaLayer(
              (attention): XLMRobertaAttention(
                (self): XLMRobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): Module

In [23]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train() # Opettele lisäämään checkpointien tallennus tähän.
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

# Kesto 55.06 GPU:lla (?)
# 1:58:54 matched datasetillä

  0%|          | 0/13062 [00:00<?, ?it/s]

In [ ]:
#import os
#os.makedirs("/content/drive/My Drive/Colab Notebooks/models") # content tästä polusta pois??
model.save_pretrained("/content/drive/My Drive/Colab Notebooks/models2") # LATAA HETI!!!
# Tämä näyttäisi toimivan
# MUTTA MISTÄ MALLI LÖYTYY DRIVESTA TÄMÄN JÄLKEEN???
# Models-kansio näkyy Colabissa muttei Drivessa
# Käytä Downloadia file explorerissa???

# Laita evaluaten tiedot ylös
# ja tarkista, tuottaako evaluate samat tulokset mallin uudelleenlatauksen jälkeen

In [25]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.8 MB/s eta 0:00:00


In [26]:
import evaluate

metric = evaluate.load("accuracy") # kts. evaluate.list_evaluation_modules() ja https://huggingface.co/evaluate-metric
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()
# accuracy 0.8552
# accuracy 0.8034

{'accuracy': 0.8034188034188035}

In [ ]:
# Siirrä tästä eteenpäin uuteen notebookiin

# Uudessa notebookissa:
# Tarkista, että accuracy alkuperäistä vastaava
# Tutki, miten saat siirrettyä ennustearvot test-dataframeen
# Jos onnistuu: miten erotella hoidolle yhteinen ja erillinen varianssi?


from transformers import AutoModelForSequenceClassification


# Lataaminen pitäisi onnistua tällä
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/My Drive/Colab Notebooks/models")

# Tarkista lataamisen jälkeen, että accuracy vastaa trainingin jälkeistä
# Pitäisi olla .6844

In [ ]:
#!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.2 MB/s eta 0:00:00


In [ ]:
import evaluate

# Huomaa, että tässäkin saattaa kestää CPU:lla kauan
metric = evaluate.load("accuracy") # kts. evaluate.list_evaluation_modules() ja https://huggingface.co/evaluate-metric
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

KeyboardInterrupt: 